In [1]:

import pygame

import math


In [2]:
print(pygame.version.ver)

1.9.2a0


In [3]:


def draw_player(x, y):
    screen.blit(player_Img, (x, y))

    
def move_player(k_p):
    pass

def draw_enemy(x, y):
    screen.blit(enemy_Img, (x, y))
    
def draw_collision(x, y):
    screen.blit(collision_Img, (x, y))
    

    
def fire_missiles(x, y):
    screen.blit(missile_Img, (x, y -20 ))
    
    

def is_collision_A_B(a_x_y, b_x_y, min_dist):
    dist = math.sqrt(sum((px - qx) ** 2.0 for px, qx in zip(a_x_y, b_x_y)))
    if dist < min_dist:
        return True
    return False

def game_over(score_stores):
    screen.fill((110,110,5))
    font = pygame.font.SysFont('arial', 30)
    stri = " GAME OVER " + "\nSCORE : "+str(score_stores)
    score = font.render(stri, True, (255,255,255))
    screen.blit(score, (screen_x/2,screen_y/2))
    pygame.display.update()
    

In [4]:

    
pygame.init()


screen_x = 900
screen_y = 800

screen = pygame.display.set_mode([screen_x, screen_y])

#All images are of size 64x64 pixels
size = 64

#PLAYER
###################
player_Img = pygame.image.load("images/rocket.png") 

player_x = 300
player_y = 770

player_x_change = 0
player_y_change = 0

player_move_speed = 0.5
###################


#ENEMY
###################
enemy_Img = pygame.image.load("images/ufo.png")


# All enemies must be correctly placed inside the borders or else weird things 
# happen.
enemy_x_change = 0.9
enemy_y_change = 64


# Put all enemies on screen in order
# Could create more patterns in which enemies show.
enemies = [ [0 + i, 100, enemy_x_change] for i in range(0, 900-64, 64)]

###################



# MISSILE
###################
missile_Img = pygame.image.load("images/missile.png")

missile_x = player_x
missile_y = player_y

missile_y_change = -1

missiles = []

###################

#collision_Img = pygame.image.load("images/supernova.png")
collision_Img = pygame.image.load("images/explosion.png")

running = True
score_stores = 0

while running:
    
    screen.fill((0, 0, 0))
    
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
            
        if event.type == pygame.KEYDOWN:
            
            
                
            if event.key == pygame.K_UP:
                player_y_change -= player_move_speed

            if event.key == pygame.K_DOWN:
                player_y_change += player_move_speed
                    
            if event.key == pygame.K_LEFT:
                player_x_change -= player_move_speed
                
            if event.key == pygame.K_RIGHT:
                player_x_change += player_move_speed
                
            if event.key == pygame.K_SPACE:
                
                missile_x = player_x
                missile_y = player_y
        
                missiles.append([missile_x, missile_y])
            
                fire_missiles(missile_x, missile_y)
                
            
        
        if event.type == pygame.KEYUP:
            #print(pygame.event.event_name(event.type))
            if event.key == pygame.K_UP or event.key == pygame.K_DOWN:
                player_y_change = 0
            if event.key == pygame.K_LEFT or event.key == pygame.K_RIGHT :
                player_x_change = 0
                
        # P is for pause game.
            if event.key == pygame.K_p:
                pygame.event.wait()
       
     
  
    # Update player movements
    player_x += player_x_change
    player_y += player_y_change
    # Check if player won.
    if len(enemies) == 0:
        game_over(score_stores)
        pygame.event.wait()
        score_stores = 0
        missiles = []
        player_x = 300
        player_y = 770
        enemies = [ [0 + i, 100, enemy_x_change] for i in range(0, 700, 64)]
        
    # MOVEMENTS & BORDERS FOR ENEMY & COLLISION WITH PLAYER
    ####################################
    for enemy in enemies:
        # Check if enemy goes all the way down, so it's game over.
        if enemy[1] >= screen_y - size:
            game_over(score_stores)
            pygame.event.wait()
            score_stores = 0
            missiles = []
            player_x = 300
            player_y = 770
            enemies = [ [0 + i, 100, enemy_x_change] for i in range(0, 700, 64)]
            break
        # Check for collision with player (1st thing that has to happen logic !)
        if is_collision_A_B([player_x, player_y], [enemy[0], enemy[1]], size + 10):
            draw_collision(player_x, player_y)
            player_x = 300
            player_y = 770
            game_over(score_stores)
            pygame.event.wait()
            score_stores = 0
            missiles = []
            player_x = 300
            player_y = 770
            enemies = [ [0 + i, 100, enemy_x_change] for i in range(0, 700, 64)]
            break
            
        
        enemy[0] += enemy[2]
        
        if enemy[0] >= screen_x - size:
            enemy[0] = screen_x - size 
            enemy[1] += size
            enemy[2] *= -1

        if enemy[0] <= 0:
            enemy[0] = 0
            enemy[1] += size
            enemy[2] *= -1
        # Finally, if all conditions are ok we can draw the enemy
        draw_enemy(enemy[0], enemy[1])
    ####################################
    ####################################
    # BORDERS FOR PLAYER and draw him 
    ####################################
    if player_y >= screen_y - size:
        player_y = screen_y - size
    if player_y <= 0:
        player_y = 0
        
    if player_x >= screen_x - size:
        player_x = screen_x - size 
    if player_x <= 0:
        player_x = 0
    # Since all is good at this point we can draw the player.
    draw_player(player_x, player_y) 
    ####################################
    ####################################
    # MOVEMENTS & BORDERS FOR MISSILE & COLLISION WITH ENEMIES
    ####################################
    for mis_idx, missile in enumerate(missiles):
        
        fire_missiles(missile[0], missile[1])
        
        
        # There is a problem where sometimes colision image won't show.
        # First thing to do is to check if there is any collision with enemy.
        for enm_idx, enemy in enumerate(enemies): 
            if is_collision_A_B([enemy[0], enemy[1]], [missile[0], missile[1]], size + 10):
                enemies.pop(enm_idx)
                missiles.pop(mis_idx)
                draw_collision(missile[0], missile[1])
                score_stores += 10
            
        missile[1] += missile_y_change
        
        if missile[1] <= 0:
             missiles.pop(mis_idx)
    ####################################
    ####################################
    # Show score
    ####################################
    font = pygame.font.SysFont('arial', 30)
    stri = "Missiles : "+str(len(missiles)) + ", Enemies : "+str(len(enemies))+", SCORE : "+str(score_stores)+", X : "+str(player_x)+", Y : "+str(player_y)
    score = font.render(stri, True, (255,255,255))
    screen.blit(score, (1,1))
    ####################################
        
    pygame.display.flip()

pygame.quit()